### Population mean - σ unknown: using scipy for confidence interval and hypothesis intervals

### Introduction
After conducting exploratory data analysis, we can use the sample to infer (or draw conclusions) about the population from which it was drawn.
Methods of making inferences about parameters are either estimating the parameter or testing a hypothesis about the value of the parameter.
A parameter is a number that describes the population (p, μ, σ) while a statistic is a number that is computed from the sample (p̂, x̄, *s*).
<br>

<br><u>*Point estimation*</u>

Point estimation is the form of statistical inference, in which, based on the sample data, we estimate the unknown parameter of interest using a single value.

<br><u>*Confidence Interval*</u>

The idea behind interval estimation is to enhance the simple point estimates by supplying information about the size of the error attached.

<br><u>*Hypothesis Testing*</u>

Statistical hypothesis testing is defined as assessing evidence provided by the data in favor of or against some claim about the population.

### Examples using scipy package

In [35]:
import numpy as np
import pandas as pd
import scipy.stats as st

**Dataset**

I will be using the [UCI](https://archive.ics.uci.edu/ml/datasets/census+income) adult census dataset -- also avaliable from [Kaggle](https://www.kaggle.com/uciml/adult-census-income).

In [2]:
df_income = pd.read_csv('data/cleaned_census_income.csv')
df_income.shape

(30162, 15)

In [3]:
df_income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30162 entries, 0 to 30161
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             30162 non-null  int64 
 1   workclass       30162 non-null  object
 2   fnlwgt          30162 non-null  int64 
 3   education       30162 non-null  object
 4   education.num   30162 non-null  int64 
 5   marital.status  30162 non-null  object
 6   occupation      30162 non-null  object
 7   relationship    30162 non-null  object
 8   race            30162 non-null  object
 9   sex             30162 non-null  object
 10  capital.gain    30162 non-null  int64 
 11  capital.loss    30162 non-null  int64 
 12  hours.per.week  30162 non-null  int64 
 13  native.country  30162 non-null  object
 14  income          30162 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.5+ MB


In [4]:
df_income.head(10)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
1,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
2,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K
3,34,Private,216864,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,3770,45,United-States,<=50K
4,38,Private,150601,10th,6,Separated,Adm-clerical,Unmarried,White,Male,0,3770,40,United-States,<=50K
5,74,State-gov,88638,Doctorate,16,Never-married,Prof-specialty,Other-relative,White,Female,0,3683,20,United-States,>50K
6,68,Federal-gov,422013,HS-grad,9,Divorced,Prof-specialty,Not-in-family,White,Female,0,3683,40,United-States,<=50K
7,45,Private,172274,Doctorate,16,Divorced,Prof-specialty,Unmarried,Black,Female,0,3004,35,United-States,>50K
8,38,Self-emp-not-inc,164526,Prof-school,15,Never-married,Prof-specialty,Not-in-family,White,Male,0,2824,45,United-States,>50K
9,52,Private,129177,Bachelors,13,Widowed,Other-service,Not-in-family,White,Female,0,2824,20,United-States,>50K


**1. Point estimate and Confidence Interval when σ is unknown**

The confidence interval for the population mean when σ is unknown is:

<img src="data/ci_mean_t.png" alt="Drawing" style="width:300px; float:left"/>

<u>Example</u>:

Find the confidence interval for the population mean of the age of adults with workclass=Private with 95% confidence.

In [7]:
income_private_sector_df = df_income[df_income.workclass == 'Private']

# number of samples
n = income_private_sector_df.age.count()
# sample mean
x_bar = income_private_sector_df.age.mean()
# sample standard deviation
s = income_private_sector_df.age.std()

{'sample_size':n, 'x_bar':x_bar, 'sample_std':s}

{'sample_size': 22286,
 'x_bar': 36.79435520057435,
 'sample_std': 12.842129264353035}

In [8]:
# determining the confidence interval

# confidence interval: C=95% and df=n-1

p_value = (1 - 0.95)/2
print('p_value for t*:', p_value)

t_multiplier = st.t.ppf(q=1-p_value, df=n-1)
print('t*:', t_multiplier)

# using stats.t.interval
ci_t = st.t.interval(alpha=0.95, df=n-1, loc=x_bar, scale=st.sem(income_private_sector_df.age))
print('Confidence interval - scipy function:', ci_t)

p_value for t*: 0.025000000000000022
t*: 1.9600704417036205
Confidence interval - scipy function: (36.625741580192035, 36.96296882095667)


**2. Hypothesis Testing**

Hypothesis testing involves four steps:

<img src="data/hyp_steps.jpg" alt="Drawing" style="width:600px; float:left"/>

The test when the standard deviation (σ) is unknown is called the **t-test** for the population mean μ.

***p-value***<br>
p-value is the probability of observing a test statistic (z-score) as extreme as that observed (or even more
extreme) assuming that the null hypothesis is true.

***t-score***<br>
Is calculated:

<img src="data/t-score.png" alt="Drawing" style="width:150px; float:left"/>

In [26]:
def print_report(p_value:float, alpha:float):
    if p_value > alpha:
        print('Since p_value(%g) > alpha(%g), the data does not provide enough evidence to reject the null hypothesis.' %(p_value, alpha))
    else:
        print('Since p_value(%g) < alpha(%g), the data provides enough evidence to reject the null hypothesis.' %(p_value, alpha))

<u>Example</u>: 

A recent study claimed that the **average age** of American adults working for the Private sector and having Income > 50K was very young in the 1990s, just about 25. The head of the US Department of Labor believes that the mean should be higher.

In [16]:
income_sub_df = df_income[(df_income.workclass=='Private') & (df_income.income=='>50K')]

# number of samples
n = income_sub_df.age.count()
# sample mean
x_bar = income_sub_df.age.mean()
# sample standard deviation
s = income_sub_df.age.std()

{'sample_size':n, 'x_bar':x_bar, 'sample_std':s}

{'sample_size': 4876,
 'x_bar': 42.8201394585726,
 'sample_std': 9.975675670762323}

In [19]:
# step 1: State the hypotheses
'''
Ho: mu = mu_0 = 25
Ha: mu > 25
'''

# step 2: collecting data and summarizing it using t-test statistics, t-score
mu_null = 25

# using stats.ttest_1samp
t_score = st.ttest_1samp(a=income_sub_df.age, popmean=mu_null, alternative='greater')
print('T-test - scipy:', t_score)
print('T-score - scipy:', t_score[0])

T-test - scipy: Ttest_1sampResult(statistic=124.73853122926442, pvalue=0.0)
T-score - scipy: 124.73853122926442


In [24]:
# step 3: finding the p-value of the test
# Ha: mu > 25
# p_value = P(t(n-1)>=t) = 1 - P(t(n-1)<=t)

print('p_value:', t_score[1])

'''
Same outcome is achieved by:
p_value = 1 - st.t.cdf(x=t_score, df=n-1)
print('p_value:', p_value)

or... sf: Survival function that calculates directly (1-CDF)
p_val_df = st.t.sf(np.abs(t_score), n-1)
print('p_value - scipy sf:', p_val_df)
'''

p_value: 0.0


In [29]:
# step 4: drawing conclusions
alpha = 0.05
p_value = t_score[1]
print_report(p_value, alpha)

Since p_value(0) < alpha(0.05), the data provides enough evidence to reject the null hypothesis.


### Another option: custom functions

In [ ]:
# confidence interval
def conf_interval(x_bar:float, s:float, t_mult:float, n:int):
    margin_error = t_mult*s/np.sqrt(n)
    return (x_bar - margin_error, x_bar + margin_error)

# t-score
def calculate_t_score(x_bar:float, mu_null:float, s:float, n:int):
    return (x_bar - mu_null)/(s/np.sqrt(n))